In [1]:
import re
import pandas as pd
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score, recall_score, precision_score
from sklearn.utils import shuffle
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
feature_type = '1'

#1 -> just sentence
#2 -> all text
#3 -> all text + other columns

In [3]:
target_type = 1

#1 -> all label
#2 -> pos + neg
#3 -> pos + neg(negative+netural)

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
train = pd.read_csv('train.csv')
test  = pd.read_csv('test.csv')

In [6]:
train.head()

,text,sentence,snp,phenotype,ASSOCIATION,CONFIDENCE
0,OBJECTIVE: Maternal smoking during pregnancy i...,We examined whether a common genetic variant a...,rs1051730,nicotine metabolism,neutral,zero
1,OBJECTIVE: Maternal smoking during pregnancy i...,We examined whether a common genetic variant a...,rs1051730,fetal growth characteristics,positive,weak
2,OBJECTIVE: Maternal smoking during pregnancy i...,RESULTS: Among mothers who did not smoke durin...,rs1051730,fetal growth characteristic,negative,-
3,OBJECTIVE: Maternal smoking during pregnancy i...,Among mothers who continued smoking during pre...,rs1051730,head circumference,negative,-
4,OBJECTIVE: Maternal smoking during pregnancy i...,The T-allele of maternal rs1051730 was associa...,rs1051730,second and third trimester fetal femur length,positive,weak


In [7]:
# dic = {"neutral": "negative"}

# train.replace({"ASSOCIATION": dic}, inplace=True)
# test.replace({"ASSOCIATION": dic}, inplace=True)

In [8]:
train['label'] = train['ASSOCIATION']
test['label']  = test['ASSOCIATION']

In [9]:
train.dropna(subset=['sentence'], inplace=True)
test.dropna(subset=['sentence'], inplace=True)

In [10]:
res = pd.read_excel('res.xlsx')

In [11]:
def vectorize_bow(df_X, cv):
    bow_X = cv.transform(df_X)
    bow_df = pd.DataFrame(data=bow_X.toarray(),
                          columns=[cv.get_feature_names_out()])
    return bow_df

def define_and_run_model(model, X_train, X_test, y_train):
    clf = model
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    return y_pred

def show_result(y_pred, y_test):
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred, digits=3))
    print(accuracy_score(y_test, y_pred))
    
def save_res(model_name, feature_type, target_type, y_pred_sr, y_test_sr, idx):
    res_pre = precision_score(y_test_sr, y_pred_sr, average='weighted')
    res_rec = recall_score(y_test_sr, y_pred_sr, average='weighted')
    res_f1 = f1_score(y_test_sr, y_pred_sr, average='weighted')
    res.at[idx, 'model-name'] = model_name
    res.at[idx, 'feature-type'] = feature_type
    res.at[idx, 'target-type'] = target_type
    res.at[idx, 'accuracy'] = accuracy_score(y_test_sr, y_pred_sr)
    res.at[idx, 'precision'] = res_pre
    res.at[idx, 'recall'] = res_rec
    res.at[idx, 'f1-score'] = res_f1

In [12]:
possible_labels = train.label.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

train['label'] = train.label.replace(label_dict)
test['label']  = test.label.replace(label_dict)

In [13]:
df = pd.concat([train, test])

In [14]:
bow_vectorizer = CountVectorizer(lowercase=False, ngram_range=(1, 1), max_df=.80, min_df=1)
bow_vectorizer.fit(df['sentence'])

CountVectorizer(lowercase=False, max_df=0.8)

In [15]:
train_bow_df  = vectorize_bow(list(train['sentence']), bow_vectorizer)
test_bow_df   = vectorize_bow(list(test['sentence']), bow_vectorizer)

In [16]:
X_train = train_bow_df
y_train = train['label']

X_test = test_bow_df
y_test = test['label']

In [17]:
X_train.shape

(934, 2902)

In [18]:
y_train.shape

(934,)

In [19]:
X_test.shape

(365, 2902)

In [20]:
y_test.shape

(365,)

In [21]:
train['ASSOCIATION'].value_counts()

positive    701
neutral     142
negative     91
Name: ASSOCIATION, dtype: int64

In [22]:
test['ASSOCIATION'].value_counts()

positive    170
neutral     166
negative     29
Name: ASSOCIATION, dtype: int64

In [23]:
%%time
smodel = SVC(kernel='rbf')
smodel.fit(X_train, y_train)
pred_values_svm = smodel.predict(X_test)
show_result(pred_values_svm, y_test)
save_res('svm', feature_type, target_type, pred_values_svm, y_test, len(res))

[[  0 160   6]
 [  0 170   0]
 [  0  27   2]]
              precision    recall  f1-score   support

           0      0.000     0.000     0.000       166
           1      0.476     1.000     0.645       170
           2      0.250     0.069     0.108        29

    accuracy                          0.471       365
   macro avg      0.242     0.356     0.251       365
weighted avg      0.242     0.471     0.309       365

0.4712328767123288
Wall time: 1.66 s


In [24]:
%%time
lr = LogisticRegression(random_state=0,
                        penalty='l2',
                        solver='newton-cg',
                        C=1.2,
                        max_iter=20)
lr.fit(X_train, y_train)
pred_values_lr = lr.predict(X_test)
show_result(pred_values_lr, y_test)
save_res('LogisticRegression', feature_type, target_type, pred_values_lr, y_test, len(res))

[[ 26 134   6]
 [  2 168   0]
 [  0  22   7]]
              precision    recall  f1-score   support

           0      0.929     0.157     0.268       166
           1      0.519     0.988     0.680       170
           2      0.538     0.241     0.333        29

    accuracy                          0.551       365
   macro avg      0.662     0.462     0.427       365
weighted avg      0.707     0.551     0.465       365

0.5506849315068493
Wall time: 578 ms


In [25]:
%%time
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train, y_train)
pred_values_rf = model.predict(X_test)
show_result(pred_values_rf, y_test)
save_res('RandomForest', feature_type, target_type, pred_values_rf, y_test, len(res))

[[  4 156   6]
 [  0 170   0]
 [  0  28   1]]
              precision    recall  f1-score   support

           0      1.000     0.024     0.047       166
           1      0.480     1.000     0.649       170
           2      0.143     0.034     0.056        29

    accuracy                          0.479       365
   macro avg      0.541     0.353     0.250       365
weighted avg      0.690     0.479     0.328       365

0.4794520547945205
Wall time: 1.48 s


In [26]:
%%time
model = KNeighborsClassifier(n_neighbors=2)
model.fit(X_train, y_train)
pred_values_knn = model.predict(X_test)
show_result(pred_values_knn, y_test)
save_res('knn', feature_type, target_type, pred_values_knn, y_test, len(res))

[[ 98  55  13]
 [ 28 141   1]
 [  3  20   6]]
              precision    recall  f1-score   support

           0      0.760     0.590     0.664       166
           1      0.653     0.829     0.731       170
           2      0.300     0.207     0.245        29

    accuracy                          0.671       365
   macro avg      0.571     0.542     0.547       365
weighted avg      0.673     0.671     0.662       365

0.6712328767123288
Wall time: 180 ms


In [27]:
%%time
model = GaussianNB()
model.fit(X_train, y_train)
pred_values_nb = model.predict(X_test)
show_result(pred_values_nb, y_test)
save_res('GaussianNB', feature_type, target_type, pred_values_nb, y_test, len(res))

[[  5 155   6]
 [  4 164   2]
 [  0  26   3]]
              precision    recall  f1-score   support

           0      0.556     0.030     0.057       166
           1      0.475     0.965     0.637       170
           2      0.273     0.103     0.150        29

    accuracy                          0.471       365
   macro avg      0.435     0.366     0.281       365
weighted avg      0.496     0.471     0.335       365

0.4712328767123288
Wall time: 195 ms


In [28]:
%%time
model = DecisionTreeClassifier(random_state=0)
model.fit(X_train, y_train)
pred_values_dt = model.predict(X_test)
show_result(pred_values_dt, y_test)
save_res('DecisionTree', feature_type, target_type, pred_values_dt, y_test, len(res))

[[ 38 124   4]
 [  4 162   4]
 [  1   7  21]]
              precision    recall  f1-score   support

           0      0.884     0.229     0.364       166
           1      0.553     0.953     0.700       170
           2      0.724     0.724     0.724        29

    accuracy                          0.605       365
   macro avg      0.720     0.635     0.596       365
weighted avg      0.717     0.605     0.549       365

0.6054794520547945
Wall time: 501 ms


In [29]:
%%time
model = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
model.fit(X_train, y_train)
pred_values_gb = model.predict(X_test)
show_result(pred_values_gb, y_test)
save_res('GradientBoosting', feature_type, target_type, pred_values_gb, y_test, len(res))

[[ 24 132  10]
 [  4 153  13]
 [  1  18  10]]
              precision    recall  f1-score   support

           0      0.828     0.145     0.246       166
           1      0.505     0.900     0.647       170
           2      0.303     0.345     0.323        29

    accuracy                          0.512       365
   macro avg      0.545     0.463     0.405       365
weighted avg      0.636     0.512     0.439       365

0.5123287671232877
Wall time: 11.8 s


In [30]:
import os
os.remove("res.xlsx") 

res.to_excel('res.xlsx', index=False)